# [AR] Contexts and their generation

Consider the case in which there are three classes of users (C1, C2, and C3), and no information about the advertising and pricing curves is known beforehand. Consider two scenarios. 

In the first one, the structure of the contexts is known beforehand. Apply the GP-UCB and GP-TS algorithms when using GPs to model the two advertising curves, reporting the plots with the average (over a sufficiently large number of runs) value and standard deviation of the cumulative regret, cumulative reward, instantaneous regret, and instantaneous reward. 

In the second scenario, the structure of the contexts is not known beforehand and needs to be learnt from data. 

Important remark: the learner does not know how many contexts there are, while it can only observe the features and data associated with the features. 

Apply the GP-UCB and GP-TS algorithms when using GPs to model the two advertising curves paired with a context generation algorithm, reporting the plots with the average (over a sufficiently large number of runs) value and standard deviation of the cumulative regret, cumulative reward, instantaneous regret, and instantaneous reward. 

Apply the context generation algorithms every two weeks of the simulation. 
Compare the performance of the two algorithms --- the one used in the first scenario with the one used in the second scenario. 

Furthermore, in the second scenario, run the GP-UCB and GP-TS algorithms without context generation, and therefore forcing the context to be only one for the entire time horizon, and compare their performance with the performance of the previous algorithms used for the second scenario.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from utils.User_Classes import UserClass
from p4.gpts_and_price_optimizer import Multi_TS_Learner

In [2]:
uc1 = UserClass(name = "C1")
uc2 = UserClass(name = "C2")
uc3 = UserClass(name = "C3")

user_classes = [uc1, uc2, uc3]

## Experiments

### First scenario

In [19]:
n_arms = 10
#Creating the pricing environment
prices = [50, 100, 150, 200, 250]

#Defining adv variables and creating the adv env
n_arms = 100
min_bid = 0.01
max_bid = 3.0
bids = np.linspace(min_bid, max_bid, n_arms)
sigma = 200

optimal_prices = [200, 200, 150]
 
T = 365

n_experiments = 1
gpts_rewards_per_experiment = []

In [17]:
#for each experiment, for each day, for each class -> different prices are considered. Let's start by considering class 0's optimal price
for e in range(n_experiments):
    multi_TS_learner = [Multi_TS_Learner(bids, prices), Multi_TS_Learner(bids, prices), Multi_TS_Learner(bids, prices)]

    for t in range(T):
        for user_class in range(len(user_classes)):
            pulled_arms = multi_TS_learner[user_class].pull_arms()
            pulled_bids_arm = pulled_arms[0]
            pulled_prices_arm = pulled_arms[1]
            round_reward = envs[user_class].round(pulled_bids_arm, pulled_prices_arm)
            multi_TS_learner[user_class].update(pulled_bids_arm, pulled_prices_arm, *round_reward)

    """
    # Store collected rewards
    ts_rewards_per_experiment.append(sum(gpts_and_price_optimizers[class_id].collected_rewards for class_id in range(n_classes)))
    ucb_rewards_per_experiment.append(sum(gpucb_and_price_optimizers[class_id].collected_rewards for class_id in range(n_classes)))

    cumregret_ts.append(np.cumsum(opt - ts_rewards_per_experiment[e]))
    cumregret_ucb.append(np.cumsum(opt - ucb_rewards_per_experiment[e]))

    cumreward_ts.append(np.cumsum(ts_rewards_per_experiment[e]))
    cumreward_ucb.append(np.cumsum(ucb_rewards_per_experiment[e]))
    """

c:\Users\Andrea\miniconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\Andrea\miniconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\Andrea\miniconda3\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified lower bound 100.0. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\Andrea\miniconda3\lib\site-packages\sklearn\gaussian_process\kernel